In [15]:
#https://susanqq.github.io/UTKFace/
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
data_path = './facedata'

#[age]_[gender]_[race]_[date&time].jpg

def load_images_from_folder(folder):
    images = []
    age_group=[]
    count0 = 0
    count2 = 0
    count1 = 0
    female = {'0':0, '1':0, '2':0}
    male = {'0':0, '1':0, '2':0}
    for filename in folder:
        img = cv2.imread(filename)
        filename = filename.split('_')
        temp = filename[0]
        temp = temp.split('/')
        try:
            age = int(temp[-1])
            gender_index = int(filename[1])
            #race_index = int(filename[2])
        except:
            continue
        if img is not None:
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img =cv2.resize(img,(224,224))
            if age<=15 and count0 <= 3500:
                count0 += 1
                age_group += [0]
                images.append(img)
                if gender_index == 0:
                    male['0'] += 1
                else:
                    female['0'] += 1
            elif age > 45 and count1 <= 3500:
                count1 += 1
                age_group+=[2]
                images.append(img)
                if gender_index == 0:
                    male['2'] += 1
                else:
                    female['2'] += 1
            elif age > 15 and age <= 45 and count2 <= 3500:
                count2 += 1
                age_group += [1]
                images.append(img)
                if gender_index == 0:
                    male['1'] += 1
                else:
                    female['1'] += 1
    return images,age_group,female,male

In [16]:
import os
import random
import sys
dir_name=sys.argv[1]
files = []
img_path = []
for filename in os.listdir(data_path):random.shuffle(files)

    files.append(filename)
# shuffle list
random.shuffle(files)
for item in files:
    img_path.append(os.path.join(data_path,item))

IndentationError: unexpected indent (<ipython-input-16-fe386c78ebd4>, line 9)

In [17]:
face_data, ages, female_count, male_count=load_images_from_folder(img_path)

In [19]:
img_data = np.array(face_data)
ages = np.array(ages)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

(10503, 224, 224, 3)

In [20]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(img_data,ages, random_state=1)

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# CNN1

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Activation
import time
from keras.utils import np_utils
import keras.callbacks as cb
from keras.optimizers import RMSprop
from keras import regularizers
from keras.optimizers import SGD
y_train = np_utils.to_categorical(y_train, 3)
y_test = np_utils.to_categorical(y_test, 3)
X_train = np.reshape(X_train, (7877, 224, 224, 3))
X_test = np.reshape(X_test, (2626, 224, 224, 3))
def init_model():
    start_time = time.time()
    print("compiling model")
    model = Sequential()
    model.add(Conv2D(96, kernel_size=(3,3), strides=(2, 2), padding='valid', activation = 'relu', input_shape=(224, 224,3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
    
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(3, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
    model.summary()
    rms = RMSprop()
    model.compile(loss = 'categorical_crossentropy', optimizer = rms, metrics=['accuracy'])
    print("Model finished" + format(time.time() - start_time))
    return model

model = init_model()

/Applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


compiling model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 111, 111, 96)      2688      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 96)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 290400)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                18585664  
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 18,588,547
Trainable params: 18,588,547
Non-trainable params: 0
_________________________________________________________________
Model finished0.1768169403076172


In [22]:
history = model.fit(X_train, y_train, batch_size = 512, epochs = 20)
score = model.evaluate(X_test, y_test, verbose=1)
print("\nNetwork's test loss and accuracy: " + format(score))

Epoch 1/20
7877/7877 [==============================] - 159s 20ms/step - loss: 10.2195 - acc: 0.3367
Epoch 2/20
7877/7877 [==============================] - 152s 19ms/step - loss: 1.3585 - acc: 0.4033
Epoch 3/20
7877/7877 [==============================] - 147s 19ms/step - loss: 1.2059 - acc: 0.4533
Epoch 4/20
7877/7877 [==============================] - 130s 16ms/step - loss: 1.1869 - acc: 0.4785
Epoch 5/20
7877/7877 [==============================] - 138s 17ms/step - loss: 1.1609 - acc: 0.4780
Epoch 6/20
7877/7877 [==============================] - 143s 18ms/step - loss: 1.1845 - acc: 0.4770
Epoch 7/20
7877/7877 [==============================] - 129s 16ms/step - loss: 1.2039 - acc: 0.4577
Epoch 8/20
7877/7877 [==============================] - 141s 18ms/step - loss: 1.1499 - acc: 0.5106
Epoch 9/20
7877/7877 [==============================] - 155s 20ms/step - loss: 1.1659 - acc: 0.4941
Epoch 10/20
7877/7877 [==============================] - 150s 19ms/step - loss: 1.1470 - acc: 0.506

In [23]:
score = model.evaluate(X_train, y_train, verbose=1)
print("\nNetwork's train loss and accuracy: " + format(score))

7877/7877 [==============================] - 61s 8ms/step

Network's train loss and accuracy: [1.1040805722608684, 0.5297702171028575]


In [24]:
# CNN 2

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Activation
import time
from keras.utils import np_utils
import keras.callbacks as cb
from keras.optimizers import RMSprop
from keras import regularizers
from keras.optimizers import SGD
y_train = np_utils.to_categorical(y_train, 3)
y_test = np_utils.to_categorical(y_test, 3)
X_train = np.reshape(X_train, (7877, 224, 224, 3))
X_test = np.reshape(X_test, (2626, 224, 224, 3))
def init_model():
    start_time = time.time()
    print("compiling model")
    model = Sequential()
    model.add(Conv2D(96, kernel_size=(3,3), strides=(2, 2), padding='valid', activation = 'relu', input_shape=(224, 224,3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))

    model.add(Conv2D(96, kernel_size=(3,3), padding='same', activation = 'relu', input_shape=(224, 224,3)))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    
    model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation = 'relu', input_shape=(224, 224,3)))
    model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation = 'relu', input_shape=(224, 224,3)))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(3, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
    model.summary()
    rms = RMSprop()
    model.compile(loss = 'categorical_crossentropy', optimizer = rms, metrics=['accuracy'])
    print("Model finished" + format(time.time() - start_time))
    return model

model = init_model()

compiling model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 111, 111, 96)      2688      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 55, 55, 96)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 55, 55, 96)        83040     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 18, 18, 96)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 18, 18, 128)       110720    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 18, 18, 64)        73792     
_________________________________________________________________
flatten_3 (Flatten)          (None, 20736)             0    

In [27]:
X_train, X_test, y_train, y_test = train_test_split(img_data,ages, random_state=1)

In [32]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 30)
score = model.evaluate(X_test, y_test, verbose=1)
print("\nNetwork's test loss and accuracy: " + format(score))

Epoch 1/30
7877/7877 [==============================] - 229s 29ms/step - loss: 1.4293 - acc: 0.3800
Epoch 2/30
7877/7877 [==============================] - 197s 25ms/step - loss: 1.1256 - acc: 0.4671
Epoch 3/30
7877/7877 [==============================] - 207s 26ms/step - loss: 1.0534 - acc: 0.5114
Epoch 4/30
7877/7877 [==============================] - 208s 26ms/step - loss: 1.0276 - acc: 0.5313
Epoch 5/30
7877/7877 [==============================] - 188s 24ms/step - loss: 0.9949 - acc: 0.5472
Epoch 6/30
7877/7877 [==============================] - 189s 24ms/step - loss: 0.9568 - acc: 0.5773
Epoch 7/30
7877/7877 [==============================] - 188s 24ms/step - loss: 0.9470 - acc: 0.5818
Epoch 8/30
7877/7877 [==============================] - 187s 24ms/step - loss: 0.9228 - acc: 0.6000
Epoch 9/30
7877/7877 [==============================] - 261s 33ms/step - loss: 0.9129 - acc: 0.6118
Epoch 10/30
7877/7877 [==============================] - 193s 25ms/step - loss: 0.8873 - acc: 0.6153

# score = model.evaluate(X_train, y_train, verbose=1)
print("\nNetwork's train loss and accuracy: " + format(score))